# 2: MTC MNL Mode Choice, Motorized

In [ ]:
# HIDDEN
import larch.numba as lx
import larch.examples
import pandas as pd
pd.set_option('display.max_columns',999)
pd.set_option('expand_frame_repr',False)
pd.set_option('display.precision',4)
larch._doctest_mode_ = True
from pytest import approx

Model 2 posits that the effect of income relative to drive alone is the same for both
shared ride modes and transit but is different for the other modes.
This is represented in the model by constraining the income coefficients in both
shared ride modes and the transit mode to be equal. [pp. 108](http://www.caee.utexas.edu/prof/Bhat/COURSES/LM_Draft_060131Final-060630.pdf)

To accomplish this, we give the income parameters for all three alternatives the same name.

In [ ]:
import larch.numba as lx
from larch import P, X, PX

d = lx.examples.MTC(format='dataset')
m = lx.Model(d)
m.title = "MTC Example 2, Motorized"

m.utility_co[2] = P("ASC_SR2")  + P("hhinc#Moto") * X("hhinc") # ←
m.utility_co[3] = P("ASC_SR3P") + P("hhinc#Moto") * X("hhinc") # ← Note same P("hhinc#Moto")
m.utility_co[4] = P("ASC_TRAN") + P("hhinc#Moto") * X("hhinc") # ←
m.utility_co[5] = P("ASC_BIKE") + P("hhinc#5") * X("hhinc")
m.utility_co[6] = P("ASC_WALK") + P("hhinc#6") * X("hhinc")

m.utility_ca = PX("tottime") + PX("totcost")

m.availability_var = 'avail'
m.choice_ca_var = 'chose'

m.ordering = (
    ("LOS", "totcost", "tottime", ),
    ("Income", "hhinc.*", ),
    ("ASCs", "ASC.*", ),
)

In [ ]:
m.maximize_loglike()

In [ ]:
# TEST
r = _
assert r.loglike == approx(-3628.2854897475563)
assert r.method == 'bhhh'
assert r.logloss == approx(0.7214725571182256)
assert r.n_cases == 5029

In [ ]:
m.calculate_parameter_covariance()

In [ ]:
# TEST
assert m.loglike() == approx(-3628.2854897475563)

In [ ]:
m.parameter_summary()

In [ ]:
# TEST
assert (m.parameter_summary().data.to_markdown()) == """
|                          |    Value |   Std Err |   t Stat | Signif   |   Null Value |
|:-------------------------|---------:|----------:|---------:|:---------|-------------:|
| ('LOS', 'totcost')       | -0.0049  |  0.000238 |   -20.57 | ***      |            0 |
| ('LOS', 'tottime')       | -0.0514  |  0.0031   |   -16.61 | ***      |            0 |
| ('Income', 'hhinc#5')    | -0.0125  |  0.00532  |    -2.35 | *        |            0 |
| ('Income', 'hhinc#6')    | -0.00925 |  0.00302  |    -3.07 | **       |            0 |
| ('Income', 'hhinc#Moto') | -0.00287 |  0.00122  |    -2.35 | *        |            0 |
| ('ASCs', 'ASC_BIKE')     | -2.39    |  0.304    |    -7.85 | ***      |            0 |
| ('ASCs', 'ASC_SR2')      | -2.14    |  0.0884   |   -24.18 | ***      |            0 |
| ('ASCs', 'ASC_SR3P')     | -3.53    |  0.115    |   -30.64 | ***      |            0 |
| ('ASCs', 'ASC_TRAN')     | -0.799   |  0.112    |    -7.11 | ***      |            0 |
| ('ASCs', 'ASC_WALK')     | -0.23    |  0.193    |    -1.19 |          |            0 |
"""[1:-1]